In [2]:
import argparse
import numpy as np

# Configurar los argumentos de línea de comandos
parser = argparse.ArgumentParser(description="Notebook para calcular suma con diferentes enfoques.")
parser.add_argument('--value', type=int, default=50000, help="Número de elementos en el array.")
args, unknown = parser.parse_known_args()

# Variable value definida por el argumento
value = args.value
print(f"Tamaño del array: {value}")


Tamaño del array: 50000


## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np
print("")
print("- Función personalizada: Lenta debido a que es secuencial y no aprovecha optimizaciones.")

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

#value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 82.4 ms ± 1.08 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24994.497684732305

Time taken by reduction operation using numpy.sum(): 105 µs ± 450 ns per loop (mean ± std. dev. of 2 runs, 10000 loops each)
Now, the result using numpy.sum(): 24994.497684732356 
 
Time taken by reduction operation using numpy.ndarray.sum(): 91.4 µs ± 1.15 µs per loop (mean ± std. dev. of 2 runs, 10000 loops each)
Now, the result using numpy.ndarray.sum(): 24994.497684732356


### a) Librería multiprocessing
Usar el paquete `multiprocessing` para acelerar la operación de reducción dividiendo el array en subarrays y procesándolos en paralelo con diferentes números de procesos.


In [4]:
from multiprocessing import Pool
print("")
print("- NumPy (sum y ndarray.sum): Muy rápido gracias a su código optimizado y vectorización.")
print("- Multiprocessing: Reduce el tiempo al aumentar los procesos porque divide el trabajo entre núcleos, pero tiene algo de sobrecarga.")

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

def parallel_sum(array, n_processes):
    """Split array and compute sum in parallel."""
    with Pool(n_processes) as pool:
        # Dividir el array en partes iguales
        subarrays = np.array_split(array, n_processes)
        # Calcular la suma de cada subarray en paralelo
        results = pool.map(reduc_operation, subarrays)
    return sum(results)

#value = 5 * 10**4
X = np.random.rand(value)

# Prueba con diferentes números de procesos
for n_processes in [1, 2, 4]:
    tiempo = %timeit -r 2 -o -q parallel_sum(X, n_processes)
    print(f"Time with {n_processes} processes: {tiempo}")
    print(f"Sum result with {n_processes} processes: {parallel_sum(X, n_processes)}\n")


- NumPy (sum y ndarray.sum): Muy rápido gracias a su código optimizado y vectorización.
- Multiprocessing: Reduce el tiempo al aumentar los procesos porque divide el trabajo entre núcleos, pero tiene algo de sobrecarga.

Time with 1 processes: 127 ms ± 251 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Sum result with 1 processes: 24990.732680867375

Time with 2 processes: 99.9 ms ± 934 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Sum result with 2 processes: 24990.732680867495

Time with 4 processes: 128 ms ± 827 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Sum result with 4 processes: 24990.732680867466



### b) Librería Numba
Optimizar la operación de reducción usando el paquete `Numba` con dos enfoques:
- `@njit` para mejorar el rendimiento en modo secuencial.
- `@njit(parallel=True)` para aprovechar el paralelismo automático.

In [3]:
from numba import njit, prange
print("")
print("- Numba (secuencial y paralelo): Excelente rendimiento. El modo paralelo es el más rápido al aprovechar múltiples núcleos y optimización a nivel de bucles.\n")

@njit
def reduc_operation_numba(A):
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

@njit(parallel=True)
def reduc_operation_parallel_numba(A):
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s

# Comparar tiempos
tiempo = %timeit -r 2 -o -q reduc_operation_numba(X)
print("Time taken by Numba (sequential):", tiempo)
print("Result:", reduc_operation_numba(X), "\n")

tiempo = %timeit -r 2 -o -q reduc_operation_parallel_numba(X)
print("Time taken by Numba (parallel):", tiempo)
print("Result:", reduc_operation_parallel_numba(X), "\n")


- Numba (secuencial y paralelo): Excelente rendimiento. El modo paralelo es el más rápido al aprovechar múltiples núcleos y optimización a nivel de bucles.




NameError: name 'X' is not defined